In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import importlib
sys.path.append(os.path.abspath("../"))

import modules
import fe_modules
import seq2seq_modules

importlib.reload(modules)
importlib.reload(fe_modules)
importlib.reload(seq2seq_modules)

from modules.memory_utils import pandas_reduce_mem_usage

In [2]:
df = pandas_reduce_mem_usage(pd.read_feather('../data/dataset_full.feather'))

Memory usage of dataframe is 9243.85 MB


  0%|          | 0/12 [00:00<?, ?it/s]

Memory usage after optimization is: 9243.85 MB
Decreased by 0.0%


In [4]:
df.columns

Index(['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
       'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'price', 'date',
       'part_of_day', 'request_cnt', 'user_id'],
      dtype='object')

In [ ]:
def un(row):
    

In [ ]:
unique= df.groupby('region_name')['cpe_model_name'].nunique().to_frame()

In [ ]:
unique.cpe_model_name.sum(), unique.cpe_model_name.shape

In [11]:
import gc

In [17]:
gc.collect()

2615